### STEP 1: Extract(Read CSV Data)
Assuming the data is coming through a CSV File

In [10]:
import os
import pandas as pd

# Load CSV files into Pandas DataFrames
advisers_df = pd.read_csv("data/Advisers.csv")
clients_df = pd.read_csv("data/Clients.csv")
transactions_df = pd.read_csv("data/Transactions.csv")

# Display the first few rows to verify data

In [11]:
print(transactions_df.head())

   TransactionID                   Type     Amount  ClientID TransactionDate  \
0              1           New Business  1000000.0         1      2025/05/01   
1              2           New Business   500000.0         2      2025/05/01   
2              3  Additional investment    10000.0         1      2025/09/01   
3              4           New Business  3000000.0         3      2025/10/01   
4              5           New Business     3000.0         4      2025/01/15   

  Currency  
0      ZAR  
1      ZAR  
2      ZAR  
3      ZAR  
4      USD  


### Step 2: Transform (Clean & Prepare Data)
Ensure the data types match those expected by SQL Server.

In [12]:

# Convert data types if necessary
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])


### Step 3: Load (Insert into SQL Server)
This loads data from CSV into the SQL Server database.

In [13]:
import pyodbc

# Database Connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MICHAELS;DATABASE=SQLAssessmentDB;Trusted_Connection=yes;')
cursor = conn.cursor()

# Function to insert data
def insert_data(table_name, df):
    for index, row in df.iterrows():
        cols = ", ".join(df.columns)
        values = ", ".join(["?" for _ in row])
        sql = f"INSERT INTO {table_name} ({cols}) VALUES ({values})"
        cursor.execute(sql, tuple(row))
    conn.commit()

# Insert data
insert_data("Advisers", advisers_df)
insert_data("Clients", clients_df)
insert_data("Transactions", transactions_df)

print("Data inserted successfully!")

# Close connection
cursor.close()
conn.close()


Data inserted successfully!


### Step 5: Export Reports to CSV
We now export these reports as CSV.

In [16]:
# Reconnect to SQL Server
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=MICHAELS;DATABASE=SQLAssessmentDB;Trusted_Connection=yes;')

# Queries for reports
queries = {
    "reports/financial_adviser_report.csv": """
        SELECT a.Name AS AdvisorName, c.Name AS ClientName, c.Telephone 
        FROM Advisers a
        JOIN Clients c ON a.AdvisorID = c.AdvisorID
        ORDER BY a.Name;
    """,
    "reports/client_transaction_report.csv": """
        SELECT c.Name AS ClientName, t.Type, t.Amount, t.TransactionDate, t.Currency
        FROM Clients c
        JOIN Transactions t ON c.ClientID = t.ClientID
        ORDER BY c.Name, t.TransactionDate;
    """
}

# Export results
for filename, query in queries.items():
    df = pd.read_sql(query, conn)
    df.to_csv(filename, index=False)

print("Reports generated successfully!")

# Close connection
conn.close()


Reports generated successfully!


C:\Users\sheph\AppData\Local\Temp\ipykernel_61416\2777939580.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
